In [47]:
import geopandas as gpd
import pandas as pd
from siuba import *

# project sources

## dropped/excluded

* US 50 (withdrawn)
* US 101 Marin (minimal accessibility/speed benefits, worthwhile project better analyzed in another context)
* MST reroute (apparent marginal/negative benefits, institutional barriers to relocating/adding service)

# read and aggregate

In [71]:
first_set = gpd.read_file('./june16_corridor_metrics_speeds.geojson')
first_set = first_set >> filter(-_.corridor.str.contains('us50')) #dropped
first_set['submission_round'] = 1

In [72]:
second_set = gpd.read_file('./sep9_corridor_metrics_speeds.geojson')
second_set['submission_round'] = 2

In [73]:
access_results = pd.read_csv('./AccessibilityResults.csv')

In [74]:
first_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,28,28,City and County of San Francisco,9.3,11.8,13.2,480.1,1690.8,56.4,159,14.9,5.3,8.5,5.6,1.1,1739.0,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
1,43,43,City and County of San Francisco,8.1,9.5,10.1,1210.7,1690.8,73.2,143,14.8,4.8,16.5,5.4,1.1,NaN,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
2,28,28,City and County of San Francisco,9.6,11.5,11.9,1061.7,1347.9,56.4,159,14.9,5.3,18.8,6.0,2.4,3850.5,16,sr1_19th_a,"POLYGON ((-217949.193 -26918.889, -217955.214 ...",1


In [75]:
second_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,4-13168,4,Los Angeles County Metropolitan Transportation...,4.7,11.1,9.3,94.2,94.2,96.2,33,2.3,7.2,1.0,7.4,1.2,1982.4,16,sr2_am,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
1,4-13168,4,Los Angeles County Metropolitan Transportation...,4.9,6.8,7.9,342.4,342.4,153.5,42,3.1,6.7,2.2,7.1,1.3,2105.6,16,sr2_pm,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
2,4-13168,4,Los Angeles County Metropolitan Transportation...,3.4,8.8,7.8,685.4,685.4,131.2,111,6.9,8.0,5.2,8.4,1.3,2105.6,16,sr2_mid,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2


In [76]:
all_corridors = pd.concat([first_set, second_set])

In [81]:
all_grouped = (all_corridors >> group_by(_.corridor, _.organization, _.submission_round)  # length and span cause fan-out, re-add later...
               >> summarize(sum_trips =_.n_trips.sum(), total_delay = _.total_speed_delay.max())
               >> mutate(avg_delay_minutes = _.total_delay / _.sum_trips)
)

## note: don't sum/aggregate accessibility results within a corridor

In [84]:
all_grouped >> left_join(_, access_results, on=['corridor', 'submission_round'])

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs
0,i805,San Diego Metropolitan Transit System,2,90,99.5,1.105556,0.015327,0.015211,0.017178,216.299637,1308.461745,0.012527,0.013475,2.518450,14.145989
1,sr123,Alameda-Contra Costa Transit District,2,219,872.0,3.981735,0.001397,0.001457,0.001271,75.015147,338.098215,0.001654,0.001523,0.953931,4.389842
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,250,3623.2,14.492800,0.003879,0.003989,0.003270,209.573414,905.891038,0.005166,0.004487,3.029337,13.415441
3,sr15,San Diego Metropolitan Transit System,2,109,313.9,2.879817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sr1_19th_a,City and County of San Francisco,1,339,1347.9,3.976106,0.003184,0.002458,0.002352,119.881855,733.304443,0.004557,0.004016,2.486017,13.624096
5,sr1_19th_b,City and County of San Francisco,1,339,956.6,2.821829,0.003184,0.002458,0.002352,119.881855,733.304443,0.004557,0.004016,2.486017,13.624096
6,sr1_lincoln,City of Santa Monica,1,115,925.9,8.051304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,sr2_am,Los Angeles County Metropolitan Transportation...,2,33,94.2,2.854545,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036
8,sr2_mid,Los Angeles County Metropolitan Transportation...,2,111,685.4,6.174775,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036
9,sr2_pm,Los Angeles County Metropolitan Transportation...,2,42,342.4,8.152381,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036


# Adding ridership (pending)

## apportioning ridership for partial-hour bus lane on sr2

* evenly during daytime hours (0500 - 2200), match post-covid trends of less peaking

In [35]:
corrs = list(all_corridors.corridor.unique())
corrs

['us101_lombard',
 'sr1_19th_a',
 'sr1_19th_b',
 'sr123_san_pablo',
 'sr1_lincoln',
 'sr2_santa_monica_bl',
 'sr82_vta',
 'sr82_samtrans',
 'sr66_foothill',
 'us101_broadway',
 'sr39_beach',
 'sr2_am',
 'sr2_pm',
 'sr2_mid',
 'sr123',
 'sr15',
 'i805']

In [ ]:
import numpy as np

In [ ]:
ridership = [14789, 12100, 12100, #19th b
            6200, 22669, 4912, #beach
            5681, 8754, 15693, #vta
            np.nan, 8200, np.nan]

In [ ]:
rider = pd.DataFrame({'corridor': corrs, 'avg_daily_riders': ridership})

ValueError: All arrays must be of the same length